- ### Addding Memory into the chat bot

- why this is importent.
    1. Our chatbot does not have the access of previous context thats why this is importent.
    2. With the previous context it limit the ability of multi-turn conversation.

- How Can we acheive this using `checkpointing`.
    1. We can provide the memory using the compilation of the graph.
    2. When calling the graph Langgraph automatically save the state after each step.
    3. When we call the graph using same `thread_id` the graph loads its saved state. allowing chatbot to pick up from there.
    4. Using checkpointing we can resume and complex state at anytime,

To Created create a Memory Saver checkpointer.

`Notice`: We are using checkpoint. This is conveniet for tutorial. In Production Application you would like to change this to use `mongo` , `postgres`.

In [3]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_core.messages import HumanMessage,AIMessage,SystemMessage

prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a professional communicator that gives responses in a professional way."),
    MessagesPlaceholder("user_input")
])

In [4]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [5]:
from dotenv import load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults

load_dotenv()

tool = TavilySearchResults(max_results=2)

tools = [tool]

In [6]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END 
from langgraph.graph.message import add_messages

from langchain_openai import ChatOpenAI

class State(TypedDict):
    messages:Annotated[list,add_messages]


graph_builder = StateGraph(State)

llm = ChatOpenAI(model="gpt-4o-mini")

llm_with_tools = llm.bind_tools(tools)


def chatbot(state:State):
    if messages := state.get("messages",{}):        
        response = llm_with_tools.invoke(messages)
        return {"messages": [response]}
    else:
        raise ValueError("Messages is not found")


graph_builder.add_node("chatbot", chatbot)

In [7]:
import json
from langchain_core.messages import ToolMessage

class BasicToolNode:
    def __init__(self,tools:list) -> None:
        self.tools_by_name = {tool.name: tool for tool in tools}

    def __call__(self, input: dict) -> dict:
        if messages := input.get("messages",[]):
            message = messages[-1]
        else:
            raise ValueError("No Message found in the input")

        outputs = []

        for tool_call in message.tool_calls:
            tool_result = self.tools_by_name[tool_call["name"]].invoke(tool_call["args"])
            
            outputs.append(
                ToolMessage(
                    content = json.dumps(tool_result),
                    name = tool_call["name"],
                    tool_call_id = tool_call["id"]
                )
            )

        return {"messages" : outputs}

tool_node = BasicToolNode(tools = tools)

graph_builder.add_node("tools", tool_node)

In [8]:
def route_tools(state: State):
    """
        Use in the conditional_edge to route to the ToolNode if the last message
        has tool calls. Otherwise, route to the end.
    """
    if isinstance(state, list):
        ai_message = state[-1]
    elif messages := state.get("messages",[]):
        ai_message = messages[-1]
    else:
        raise ValueError(f"No message found in input state to tool_edge: {state}")
    
    if hasattr(ai_message,"tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"
    
    return END

graph_builder.add_conditional_edges(
    "chatbot",
    route_tools,
    {"tools":"tools",END:END}
)

# EDGES DEFINE THE ORDER FOR DECISIONS --> LIKE BELOW 
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

graph = graph_builder.compile(checkpointer=memory)

In [9]:

config = {"configurable" : {"thread_id" : "2"}}

def call_chat_bot(user_input:str):
    # for events in graph.stream({"messages": user_input}):
    #     for value in events.values():
    #         if "messages" in value:
    #             print("Assistant: ",value["messages"][-1].content)

    events = graph.stream({"messages": [{"role": "user", "content": user_input}]},config,
    stream_mode="values",)
    
    for event in events:
        event["messages"][-1].pretty_print()

while True:
    user_input = input("User: ")

    if user_input.lower() in ["quit", "q", "exit"]:
        break
    
    call_chat_bot(user_input)

================================ Human Message =================================

what is the capital of india.
================================== Ai Message ==================================

The capital of India is New Delhi.
================================ Human Message =================================

whats its population.
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_C4e8lGomuQ0BLF68AhWVBfvx)
 Call ID: call_C4e8lGomuQ0BLF68AhWVBfvx
  Args:
    query: population of New Delhi 2023
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.macrotrends.net/cities/21228/delhi/population", "content": "The metro area population of Delhi in 2023 was 32,941,000, a 2.73% increase from 2022. The metro area population of Delhi in 2022 was 32,066,000, a 2.84%"}, {"url": "https://worldpopulationreview.com/cities/india/delhi", "conten